In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import Adam, SGD, Adadelta, Adagrad
from keras import Sequential, Model
from keras.models import load_model
from keras.utils import np_utils
from keras.metrics import Accuracy
from keras.preprocessing.image import ImageDataGenerator  
from keras.layers import (
    GlobalAveragePooling2D, Multiply, Flatten, Input,Lambda,
     Dense, Dropout, Conv2D, BatchNormalization, MaxPooling2D, Resizing)
from keras.callbacks import EarlyStopping
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras import activations
from vis.utils import utils

%pylab inline
import cv2
import glob
import os
import matplotlib.pylab as plt
import pickle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import random
import seaborn as sns 
import shutil 
import time
import numpy as np

import pandas as pd

In [ ]:
#Funcion que genera los conjuntos de train y test de una carpeta concreta
def get_data(path, bt = 10):  
  train_generator = ImageDataGenerator(rotation_range=20,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       horizontal_flip=True,
                                       zoom_range = 0.2,
                                       shear_range = 0.2,
                                       vertical_flip = True ,
                                       preprocessing_function=preprocess_input)
  
  
  test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


  path_train = path + '/train'
  train_data = train_generator.flow_from_directory(directory= path_train,
                                                      class_mode='categorical',
                                                      shuffle = True, 
                                                      batch_size = bt,
                                                      seed=21,
                                                      classes=['Normal', 'Fracture'], 
                                                      target_size = (224,224)
                                                      )
  
  path_test = path + '/test'
  test_data = test_generator.flow_from_directory(directory= path_test,
                                                      class_mode='categorical',
                                                      shuffle = False,
                                                      batch_size = bt,
                                                      seed = 21,
                                                      classes=['Normal', 'Fracture'],
                                                      target_size = (224,224),
                                                      )
  
  return train_data, test_data

In [ ]:
#Definicion modelo 
def get_model():

  nn = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
  nn.trainable = False #congelar valores

  model = Sequential()
  model.add(nn)
  model.add(Flatten(name = "fla"))
  model.add(Dropout(0.5, name = "dro")) 
  model.add(Dense(2, activation = "softmax",name='predictions'))

  return model

In [ ]:
#Definicion funcion que entrena el modelo
def train_model(model, data,path_model, name, optimizer_type, learning_rate, batch_size): 

  opt = optimizer_type(learning_rate=learning_rate)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

  cb = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose = 0)
  
  history_model = model.fit(data,
                  epochs=10,
                  batch_size = batch_size,
                  callbacks=[cb],
                  verbose = 0)

     
  model.save(path_model + 'weights_model/' + name + '.pkl')
  metrics=history_model.history
  f = open(path_model + 'metrics_model/' + name + '.pkl',"wb")
  pickle.dump(metrics,f)
  f.close()

  return model

In [ ]:
# funcion que calcula el accuracy del modelo
def model_accuracy(model, data):
  y_pred = model.predict(data)
  y_pred = np.argmax(y_pred, axis=1)

  tn, fp, fn, tp =confusion_matrix(y_true =data.classes,y_pred =y_pred).ravel()
  return round((tn+tp)/(tn+fp+fn+tp),4)

In [ ]:
#Declaracion de variables y entrenamiento de todos los modelos posibles
OPTIMIZERS =[Adam, SGD, Adadelta, Adagrad]
LEARNING_RATES = [0.001,0.01, 0.05,0.1]
BATCH_SIZE = [10,20,30,40]

#nombres de las variables para guardar el modelo
OPTIMIZERS_NAME =["Adam", "SGD", "Adadelta", "Adagrad"]
LEARNING_RATES_NAME = ["0,001","0,01", "0,05","0,1"]

#vectores para guardar resultados
results_full = []
results_A = []
results_L = []
acc_full_max = 0
acc_A_max = 0
acc_L_max = 0
                   
i = 0
for op in OPTIMIZERS:
    j = 0
    for lr in LEARNING_RATES:
      k = 0
      for bt in BATCH_SIZE:

        #cargar datos con el batch size apropiado
        train_full, test_full = get_data('/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture', bt = bt)
        train_A, test_A = get_data('/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture A View', bt = bt)
        train_L, test_L = get_data('/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture L View', bt = bt)

        #creacion del model + entrenamiento
        model_name =  OPTIMIZERS_NAME[i] + '_' + LEARNING_RATES_NAME[j] + '_' +str(bt)

        model = get_model()
        model_full = train_model(model = model,data = train_full,path_model = '/content/drive/MyDrive/TFM/PAC3_Modelos/Modelo_final/01_full_model/' ,name = model_name, optimizer_type = op,learning_rate = lr,batch_size = bt)
        model = get_model()
        model_A = train_model(model = model,data = train_A,path_model = '/content/drive/MyDrive/TFM/PAC3_Modelos/Modelo_final/02_A_model/' ,name = model_name, optimizer_type = op,learning_rate = lr,batch_size = bt)
        model = get_model()
        model_L = train_model(model = model,data = train_L,path_model = '/content/drive/MyDrive/TFM/PAC3_Modelos/Modelo_final/03_L_model/' ,name = model_name, optimizer_type = op,learning_rate = lr,batch_size = bt)

        #evaludar modelo
        acc_full_model = model_accuracy(model_full,test_full)
        acc_A_model = model_accuracy(model_A,test_A)
        acc_L_model = model_accuracy(model_L,test_L)     
        
        #guardar valors      
        results_full.append([OPTIMIZERS_NAME[i],LEARNING_RATES[j],BATCH_SIZE[k],acc_full_model])
        results_A.append([OPTIMIZERS_NAME[i],LEARNING_RATES[j],BATCH_SIZE[k],acc_A_model])
        results_L.append([OPTIMIZERS_NAME[i],LEARNING_RATES[j],BATCH_SIZE[k],acc_L_model])
        k += 1
      j += 1
    i += 1

#guardar resultatos
results_full = pd.DataFrame(results_full, columns =["optimizer", "learning_rate","batch_size", 'accuracy']) 
results_full.to_csv("/content/drive/MyDrive/TFM/PAC3_Modelos/Modelo_final/results/GS_full.csv",sep=';')

results_A = pd.DataFrame(results_A, columns =["optimizer", "learning_rate","batch_size", 'accuracy'])
results_A.to_csv("/content/drive/MyDrive/TFM/PAC3_Modelos/Modelo_final/results/GS_A.csv", sep=';')

results_L = pd.DataFrame(results_L, columns =["optimizer", "learning_rate","batch_size", 'accuracy'])
results_L.to_csv("/content/drive/MyDrive/TFM/PAC3_Modelos/Modelo_final/results/GS_L.csv", sep=';')